# 1. Data Cleaning and Preparation

## Dataset: HDB resale prices since 2017

Resale transacted prices since 2017 with attributes. The data is based on date of registration for the resale transactions.

Housing and Development Board (HDB). (2024). Resale flat prices based on registration date from Jan-2017 onwards [Data set]. https://beta.data.gov.sg/collections/189/datasets/d_8b84c4ee58e3cfc0ece0d773c8ca6abc/view

This notebook outlines the steps taken to ensure that we are able to obtain meaningful insights from the dataset to address our question.

Summary:
1. Convert variables to workable format
2. Add coordinate data for addresses
3. Dropping NaNs, outliers and irrelevant columns
4. Creating new features


In [ ]:
df_resale = pd.read_csv('ResaleflatpricesbasedonregistrationdatefromJan2017onwards.csv')
df_coordinate = pd.read_excel('coordinates.xlsm')
df_resale.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0


In [ ]:
df_resale.shape

(174995, 11)

In [ ]:
df_resale.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174995 entries, 0 to 174994
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   month                174995 non-null  object 
 1   town                 174995 non-null  object 
 2   flat_type            174995 non-null  object 
 3   block                174995 non-null  object 
 4   street_name          174995 non-null  object 
 5   storey_range         174995 non-null  object 
 6   floor_area_sqm       174995 non-null  float64
 7   flat_model           174995 non-null  object 
 8   lease_commence_date  174995 non-null  int64  
 9   remaining_lease      174995 non-null  object 
 10  resale_price         174995 non-null  float64
dtypes: float64(2), int64(1), object(8)
memory usage: 14.7+ MB


We noticed that some variables were not in the correct Dtype eg. month and remaining lease

## 1. Convert variables to workable format

We converted 'month' to date time format for easier manipulation of data, convert 'remaining_lease' into numeric variable such that we can work with it easier. 

In [ ]:
df_resale["month"] = pd.to_datetime(df_resale["month"])
df_resale["lease_commence_date"] = pd.to_datetime(df_resale["lease_commence_date"])
# Define a function to convert the string into total number of months
def convert_to_years(s):
    try:
        # Split the string into years and months
        parts = s.split(' years ')
        if len(parts) == 2:  # Case: Years and months are both specified
            years_str, months_str = parts
            months_parts = months_str.split()
            # Extract the numeric part of months and convert it to an integer
            months = int(months_parts[0]) if months_parts else 0
            # Calculate the total number of years including the fraction of months
            decimal_years = int(years_str) + months / 12
            return round(decimal_years,2)
        elif len(parts) == 1:  # Case: Only years are specified
            years_parts = parts[0].split()
            if years_parts:
                return int(years_parts[0])
            else:
                return None
        else:
            # If the string format is unexpected, log a warning and return None
            print("Warning: Unexpected input format:", s)
            return None
    except ValueError as e:
        # Handle the case where the string doesn't have the expected format
        # Log the error for debugging purposes
        print("Error:", e)
        # Return None or another appropriate default value
        return None

# Specify the name of the column you want to convert
column_to_convert = 'remaining_lease'

# Apply the function to the column to convert it into numeric data
df_resale['remaininglease'] = df_resale[column_to_convert].apply(convert_to_years)

## 2. Add coordinate data for addresses

As we would like to see how location affects resale_price, having coordinates would be useful. We do this by matching the address with coordinate data we obtained.

In [ ]:
#create new column 'address' which combines blk and st
blk = 'block'
st = 'street_name'
df_resale['address'] = df_resale[blk] + ' ' + df_resale[st]

In [ ]:
df_resale = pd.merge(df_resale, df_coordinate[['address', 'latitude', 'longitude']], 
                     how='left', left_on='address', right_on='address')

Add Coordinate data 

In [ ]:
df_resale.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,remaininglease,address,latitude,longitude
0,2017-01-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1970-01-01 00:00:00.000001979,61 years 04 months,232000.0,61.33,406 ANG MO KIO AVE 10,1.362005,103.853880
1,2017-01-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1970-01-01 00:00:00.000001978,60 years 07 months,250000.0,60.58,108 ANG MO KIO AVE 4,1.370966,103.838202
2,2017-01-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1970-01-01 00:00:00.000001980,62 years 05 months,262000.0,62.42,602 ANG MO KIO AVE 5,1.380709,103.835368
3,2017-01-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1970-01-01 00:00:00.000001980,62 years 01 month,265000.0,62.08,465 ANG MO KIO AVE 10,1.366201,103.857201
4,2017-01-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1970-01-01 00:00:00.000001980,62 years 05 months,265000.0,62.42,601 ANG MO KIO AVE 5,1.381041,103.835132


## 3. Dropping NaNs, outliers and irrelevant columns

We dropped the entries with NaNs in our dataset

In [ ]:
total_na = df_resale.isna().sum().sum()
print(f"Total number of NaN values in the DataFrame: {total_na}")

Total number of NaN values in the DataFrame: 5880


In [ ]:
columns_to_check = ['latitude']

# Use boolean indexing to filter rows where NaN values occur in the specified column(s)
rows_with_nan = df_resale[df_resale[columns_to_check].isna().any(axis=1)]

# Now rows_with_nan contains the rows where NaN values occur in the specified column(s)
print(rows_with_nan)

            month           town  flat_type block         street_name  \
315    2017-01-01  CHOA CHU KANG     4 ROOM   121      TECK WHYE LANE   
327    2017-01-01  CHOA CHU KANG     4 ROOM   239  CHOA CHU KANG CTRL   
412    2017-01-01        GEYLANG     5 ROOM   134  GEYLANG EAST AVE 1   
482    2017-01-01    JURONG EAST     3 ROOM    37       TEBAN GDNS RD   
516    2017-01-01    JURONG EAST  EXECUTIVE    45       TEBAN GDNS RD   
...           ...            ...        ...   ...                 ...   
174785 2024-03-01       TAMPINES     4 ROOM  606B      TAMPINES ST 61   
174786 2024-03-01       TAMPINES     4 ROOM  606A      TAMPINES ST 61   
174797 2024-03-01       TAMPINES     5 ROOM  601A      TAMPINES AVE 9   
174967 2024-03-01         YISHUN     4 ROOM   292        YISHUN ST 22   
174973 2024-03-01         YISHUN     4 ROOM  501A        YISHUN ST 51   

       storey_range  floor_area_sqm  flat_model           lease_commence_date  \
315        01 TO 03           103.0     Mo

In [ ]:
filtered_df = df_resale.dropna(how = 'any')
filtered_df

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,remaininglease,address,latitude,longitude
0,2017-01-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1970-01-01 00:00:00.000001979,61 years 04 months,232000.0,61.33,406 ANG MO KIO AVE 10,1.362005,103.853880
1,2017-01-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1970-01-01 00:00:00.000001978,60 years 07 months,250000.0,60.58,108 ANG MO KIO AVE 4,1.370966,103.838202
2,2017-01-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1970-01-01 00:00:00.000001980,62 years 05 months,262000.0,62.42,602 ANG MO KIO AVE 5,1.380709,103.835368
3,2017-01-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1970-01-01 00:00:00.000001980,62 years 01 month,265000.0,62.08,465 ANG MO KIO AVE 10,1.366201,103.857201
4,2017-01-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1970-01-01 00:00:00.000001980,62 years 05 months,265000.0,62.42,601 ANG MO KIO AVE 5,1.381041,103.835132
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174990,2024-03-01,YISHUN,5 ROOM,607,YISHUN ST 61,01 TO 03,121.0,Improved,1970-01-01 00:00:00.000001987,62 years 07 months,613000.0,62.58,607 YISHUN ST 61,1.421386,103.835604
174991,2024-03-01,YISHUN,5 ROOM,841,YISHUN ST 81,07 TO 09,122.0,Improved,1970-01-01 00:00:00.000001988,62 years 11 months,745000.0,62.92,841 YISHUN ST 81,1.416013,103.834369
174992,2024-03-01,YISHUN,EXECUTIVE,344,YISHUN AVE 11,01 TO 03,145.0,Maisonette,1970-01-01 00:00:00.000001988,63 years 05 months,820000.0,63.42,344 YISHUN AVE 11,1.429031,103.844303
174993,2024-03-01,YISHUN,EXECUTIVE,791,YISHUN AVE 2,10 TO 12,146.0,Apartment,1970-01-01 00:00:00.000001987,62 years 07 months,851888.0,62.58,791 YISHUN AVE 2,1.420520,103.833564


Finding outliers

In [ ]:
def detect_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers

outliers_iqr = detect_outliers_iqr(filtered_df, 'resale_price')
print("Number of outliers detected using IQR method:", len(outliers_iqr))
print("\nOutliers using IQR Method:")
print(outliers_iqr)

Number of outliers detected using IQR method: 3910

Outliers using IQR Method:
            month             town  flat_type block         street_name  \
310    2017-01-01     CENTRAL AREA     5 ROOM    1B       CANTONMENT RD   
664    2017-01-01  KALLANG/WHAMPOA     5 ROOM     7        BOON KENG RD   
1329   2017-02-01           BISHAN     5 ROOM  273B        BISHAN ST 24   
1397   2017-02-01      BUKIT MERAH     5 ROOM    56         HAVELOCK RD   
1398   2017-02-01      BUKIT MERAH     5 ROOM    53         HAVELOCK RD   
...           ...              ...        ...   ...                 ...   
174836 2024-03-01        TOA PAYOH     4 ROOM  138A    LOR 1A TOA PAYOH   
174841 2024-03-01        TOA PAYOH     4 ROOM   261      TOA PAYOH EAST   
174842 2024-03-01        TOA PAYOH     5 ROOM   121  POTONG PASIR AVE 1   
174914 2024-03-01        WOODLANDS  EXECUTIVE   302     WOODLANDS ST 31   
174915 2024-03-01        WOODLANDS  EXECUTIVE   835     WOODLANDS ST 83   

       storey_range 

We have decided not to remove any outliers as we want to understand the full range of resale prices in the housing market, removing outliers might remove any uniquely positioned flats or luxury units in prime locations.

Removing any irrelevant columns

In [ ]:
filtered_df.drop(columns = "street_name", inplace = True) ##maybe keep? 
filtered_df.drop(columns = "block", inplace = True)
filtered_df.drop(columns = "remaining_lease", inplace = True)


filtered_df

/shared-libs/python3.8/py/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,month,town,flat_type,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaininglease,address,latitude,longitude
0,2017-01-01,ANG MO KIO,2 ROOM,10 TO 12,44.0,Improved,1970-01-01 00:00:00.000001979,232000.0,61.33,406 ANG MO KIO AVE 10,1.362005,103.853880
1,2017-01-01,ANG MO KIO,3 ROOM,01 TO 03,67.0,New Generation,1970-01-01 00:00:00.000001978,250000.0,60.58,108 ANG MO KIO AVE 4,1.370966,103.838202
2,2017-01-01,ANG MO KIO,3 ROOM,01 TO 03,67.0,New Generation,1970-01-01 00:00:00.000001980,262000.0,62.42,602 ANG MO KIO AVE 5,1.380709,103.835368
3,2017-01-01,ANG MO KIO,3 ROOM,04 TO 06,68.0,New Generation,1970-01-01 00:00:00.000001980,265000.0,62.08,465 ANG MO KIO AVE 10,1.366201,103.857201
4,2017-01-01,ANG MO KIO,3 ROOM,01 TO 03,67.0,New Generation,1970-01-01 00:00:00.000001980,265000.0,62.42,601 ANG MO KIO AVE 5,1.381041,103.835132
...,...,...,...,...,...,...,...,...,...,...,...,...
174990,2024-03-01,YISHUN,5 ROOM,01 TO 03,121.0,Improved,1970-01-01 00:00:00.000001987,613000.0,62.58,607 YISHUN ST 61,1.421386,103.835604
174991,2024-03-01,YISHUN,5 ROOM,07 TO 09,122.0,Improved,1970-01-01 00:00:00.000001988,745000.0,62.92,841 YISHUN ST 81,1.416013,103.834369
174992,2024-03-01,YISHUN,EXECUTIVE,01 TO 03,145.0,Maisonette,1970-01-01 00:00:00.000001988,820000.0,63.42,344 YISHUN AVE 11,1.429031,103.844303
174993,2024-03-01,YISHUN,EXECUTIVE,10 TO 12,146.0,Apartment,1970-01-01 00:00:00.000001987,851888.0,62.58,791 YISHUN AVE 2,1.420520,103.833564


## 4. Creating new features

We have decided to convert storey range, a categorical variable into a numerical variable by taking the midpoint of every range

In [ ]:
filtered_df['storey_midpoint'] = filtered_df['storey_range'].apply(lambda x: (int(x.split(' TO ')[0]) + int(x.split(' TO ')[1])) / 2)

/tmp/ipykernel_45/4090472005.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['storey_midpoint'] = filtered_df['storey_range'].apply(lambda x: (int(x.split(' TO ')[0]) + int(x.split(' TO ')[1])) / 2)


We also converted the date and time into a numerical variable in months, starting from the base month 2017 January, taking it to be 1.

Example: 2017 Jan = 1

2018 Jan = 13

In [ ]:
base_date = pd.Timestamp('2017-01-01')
filtered_df['numerical_month'] = (filtered_df['month'].dt.year - base_date.year) * 12 + (filtered_df['month'].dt.month - base_date.month + 1)


/tmp/ipykernel_45/1510092436.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['numerical_month'] = (filtered_df['month'].dt.year - base_date.year) * 12 + (filtered_df['month'].dt.month - base_date.month + 1)


We also decided to calculate and include distance to city centre

The latitude of Downtown Core, Singapore is 1.287953, and the longitude is 103.851784. Downtown Core, Singapore is located at Singapore country in the Districts place category with the gps coordinates of 1° 17' 16.6308'' N and 103° 51' 6.4224'' E.

Below is the Haversine formula to calculate distance from the flat to city centre.

In [ ]:
import math

def haversine(lat1, lon1, lat2, lon2):
    # convert decimal degrees to radians 
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    
    # Haversine formula 
    dlat = lat2 - lat1 
    dlon = lon2 - lon1 
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a)) 
    
    # Radius of earth in kilometers. Use 3956 for miles
    r = 6371 
    
    return c * r

In [ ]:
#city centre The latitude of Downtown Core, Singapore is 1.287953, and the longitude is 103.851784. Downtown Core, Singapore is located at Singapore country in the Districts place category with the gps coordinates of 1° 17' 16.6308'' N and 103° 51' 6.4224'' E.
lat1 = 1.287953
lon1 = 103.851784
filtered_df['Distance_from_Centre (km)'] = filtered_df.apply(lambda row: haversine(lat1, lon1, row['latitude'], row['longitude']), axis=1)
filtered_df.head()

/tmp/ipykernel_45/1279185763.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Distance_from_Centre (km)'] = filtered_df.apply(lambda row: haversine(lat1, lon1, row['latitude'], row['longitude']), axis=1)


,month,town,flat_type,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaininglease,address,latitude,longitude,storey_midpoint,numerical_month,Distance_from_Centre (km),distance_to_nearest_mrt,distance_to_nearest_mall
0,2017-01-01,ANG MO KIO,2 ROOM,10 TO 12,44.0,Improved,1970-01-01 00:00:00.000001979,232000.0,61.33,406 ANG MO KIO AVE 10,1.362005,103.853880,11.0,1,8.237451,940.433589,1024.343762
1,2017-01-01,ANG MO KIO,3 ROOM,01 TO 03,67.0,New Generation,1970-01-01 00:00:00.000001978,250000.0,60.58,108 ANG MO KIO AVE 4,1.370966,103.838202,2.0,1,9.353331,201.370349,864.215691
2,2017-01-01,ANG MO KIO,3 ROOM,01 TO 03,67.0,New Generation,1970-01-01 00:00:00.000001980,262000.0,62.42,602 ANG MO KIO AVE 5,1.380709,103.835368,2.0,1,10.474170,491.898968,1558.616517
3,2017-01-01,ANG MO KIO,3 ROOM,04 TO 06,68.0,New Generation,1970-01-01 00:00:00.000001980,265000.0,62.08,465 ANG MO KIO AVE 10,1.366201,103.857201,5.0,1,8.721599,894.428774,898.970693
4,2017-01-01,ANG MO KIO,3 ROOM,01 TO 03,67.0,New Generation,1970-01-01 00:00:00.000001980,265000.0,62.42,601 ANG MO KIO AVE 5,1.381041,103.835132,2.0,1,10.515178,462.994532,1603.405761


We also decided to calculate the distances to the closest shopping mall and MRT Station to see the effect of proximity to such amenities.

In order to search more efficiently, we used a KD-Tree

MRT Station coordinates

Shopping Mall coordinates

In [ ]:
mrt = pd.read_csv('MRT Stations.csv') #Caps
mall = pd.read_csv('/work/shopping_mall_coordinates.csv') #ALLCAPS

In [ ]:
from scipy.spatial import cKDTree

# Function to convert latitude and longitude to approximate meters
def latlon_to_meters(lat, lon):
    lat_m = lat * 110574  # Approximate length of a degree of latitude in meters
    lon_m = lon * 111320 * np.cos(np.radians(lat))  # Approximate length of a degree of longitude in meters
    return lat_m, lon_m

# Load data

mrt_stations = pd.read_csv('MRT Stations.csv')
mall = pd.read_csv('/work/shopping_mall_coordinates.csv')

# Prepare coordinates for KD-Tree in meter approximation
filtered_coords = np.array([latlon_to_meters(lat, lon) for lat, lon in zip(filtered_df['latitude'], filtered_df['longitude'])])
mrt_coords = np.array([latlon_to_meters(lat, lon) for lat, lon in zip(mrt_stations['Latitude'], mrt_stations['Longitude'])])
mall_coords = np.array([latlon_to_meters(lat, lon) for lat, lon in zip(mall['LATITUDE'], mall['LONGITUDE'])])

# Create KD-Tree for MRT stations
mrt_tree = cKDTree(mrt_coords)
mall_tree = cKDTree(mall_coords)

# Query KD-Tree for nearest MRT station distance for each property
distances_mrt, _ = mrt_tree.query(filtered_coords, k=1)  # distances are in meters
distances_mall, _ = mall_tree.query(filtered_coords, k=1)  # distances are in meters


# Assign calculated distances to the DataFrame
filtered_df['distance_to_nearest_mrt'] = distances_mrt
filtered_df['distance_to_nearest_mall'] = distances_mall


# Save or display the updated DataFrame

filtered_df.head()

/tmp/ipykernel_45/3085883054.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['distance_to_nearest_mrt'] = distances_mrt
/tmp/ipykernel_45/3085883054.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['distance_to_nearest_mall'] = distances_mall


,month,town,flat_type,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaininglease,address,latitude,longitude,storey_midpoint,numerical_month,Distance_from_Centre (km),distance_to_nearest_mrt,distance_to_nearest_mall
0,2017-01-01,ANG MO KIO,2 ROOM,10 TO 12,44.0,Improved,1970-01-01 00:00:00.000001979,232000.0,61.33,406 ANG MO KIO AVE 10,1.362005,103.853880,11.0,1,8.237451,940.433589,1024.343762
1,2017-01-01,ANG MO KIO,3 ROOM,01 TO 03,67.0,New Generation,1970-01-01 00:00:00.000001978,250000.0,60.58,108 ANG MO KIO AVE 4,1.370966,103.838202,2.0,1,9.353331,201.370349,864.215691
2,2017-01-01,ANG MO KIO,3 ROOM,01 TO 03,67.0,New Generation,1970-01-01 00:00:00.000001980,262000.0,62.42,602 ANG MO KIO AVE 5,1.380709,103.835368,2.0,1,10.474170,491.898968,1558.616517
3,2017-01-01,ANG MO KIO,3 ROOM,04 TO 06,68.0,New Generation,1970-01-01 00:00:00.000001980,265000.0,62.08,465 ANG MO KIO AVE 10,1.366201,103.857201,5.0,1,8.721599,894.428774,898.970693
4,2017-01-01,ANG MO KIO,3 ROOM,01 TO 03,67.0,New Generation,1970-01-01 00:00:00.000001980,265000.0,62.42,601 ANG MO KIO AVE 5,1.381041,103.835132,2.0,1,10.515178,462.994532,1603.405761


Lastly, we convert the dataframe filtered_df into a csv for us to work on 

In [ ]:
filtered_df.to_csv('filtered_df.csv', index=False)
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 172055 entries, 0 to 174994
Data columns (total 17 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   month                      172055 non-null  datetime64[ns]
 1   town                       172055 non-null  object        
 2   flat_type                  172055 non-null  object        
 3   storey_range               172055 non-null  object        
 4   floor_area_sqm             172055 non-null  float64       
 5   flat_model                 172055 non-null  object        
 6   lease_commence_date        172055 non-null  datetime64[ns]
 7   resale_price               172055 non-null  float64       
 8   remaininglease             172055 non-null  float64       
 9   address                    172055 non-null  object        
 10  latitude                   172055 non-null  float64       
 11  longitude                  172055 non-null  float64 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d45d76c1-fda9-4cb4-b314-47c76fd050a8' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>